<a href="https://colab.research.google.com/github/MahdiEsrafili/pert/blob/master/ezafe_blstm_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install stanza

     |████████████████████████████████| 432 kB 37.3 MB/s 
     |████████████████████████████████| 175 kB 69.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=31679dd73b67b98c3091f21bbff66aa08e14fd7a3820c274f9384924bd9529cd
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [54]:
!pip3 install parsivar

     |████████████████████████████████| 36.2 MB 222 kB/s 
     |████████████████████████████████| 1.5 MB 39.6 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492972 sha256=3ce6bc036449d6fdc3be47dea091b5dd07beeb2e7fe88031158f5c18214f3af9
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449922 sha256=95075ab17ce78f35e6bd683661d6d1b102862210e87d329b557489198b113ddf
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [1]:
import stanza
stanza.download('fa')

2022-04-19 10:36:21 INFO: Downloading default packages for language: fa (Persian)...
2022-04-19 10:36:22 INFO: File exists: /root/stanza_resources/fa/default.zip.
2022-04-19 10:36:25 INFO: Finished downloading models and saved to /root/stanza_resources.


In [2]:
nlp_pos = stanza.Pipeline(lang='fa', processors='tokenize,mwt,pos')

2022-04-19 10:36:27 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |

2022-04-19 10:36:27 INFO: Use device: gpu
2022-04-19 10:36:27 INFO: Loading: tokenize
2022-04-19 10:36:31 INFO: Loading: mwt
2022-04-19 10:36:31 INFO: Loading: pos
2022-04-19 10:36:31 INFO: Done loading processors!


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
% cd fastText
!  pip install .
%cd ..

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 3930 (delta 28), reused 42 (delta 11), pack-reused 3854
Receiving objects: 100% (3930/3930), 8.33 MiB | 38.27 MiB/s, done.
Resolving deltas: 100% (2445/2445), done.
/content/fastText
Processing /content/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3139533 sha256=98cc7ca5d1af18910d7b21d5387950286ea748c6c5a497c0ee1f8

In [ ]:
% cd fastText
!./download_model.py fa
% cd ..

/content/fastText
 (100.00%) [==================================================>]
/content


In [5]:
!mkdir data

In [6]:
# run this or below cell
!cp drive/MyDrive/دیتاست\ بیجن\ خان/test_clean.tsv data/test_clean.tsv
!cp drive/MyDrive/دیتاست\ بیجن\ خان/train_clean.tsv data/train_clean.tsv

In [ ]:
!wc -l data/train_clean.tsv

9049352 data/train_clean.tsv


In [ ]:
!wc -l data/test_clean.tsv

1601230 data/test_clean.tsv


In [15]:
!head -n4000000 data/train_clean.tsv > data/train_clean_p1.tsv

In [ ]:
!wc -l data/train_clean_p1.tsv

4000000 data/train_clean_p1.tsv


In [ ]:
!unzip drive/MyDrive/دیتاست\ بیجن\ خان/test_data.zip -d data
!unzip drive/MyDrive/دیتاست\ بیجن\ خان/train_data.zip -d data

Archive:  drive/MyDrive/دیتاست بیجن خان/test_data.zip
  inflating: data/test_data.txt      
Archive:  drive/MyDrive/دیتاست بیجن خان/train_data.zip
  inflating: data/train_data.txt     


In [3]:
nlp_pos = stanza.Pipeline(lang='fa', processors='tokenize,mwt,pos')

2022-04-19 08:20:45 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |

2022-04-19 08:20:45 INFO: Use device: gpu
2022-04-19 08:20:45 INFO: Loading: tokenize
2022-04-19 08:20:45 INFO: Loading: mwt
2022-04-19 08:20:45 INFO: Loading: pos
2022-04-19 08:20:45 INFO: Done loading processors!


In [5]:
nlp_pos = stanza.Pipeline(lang='fa', processors='tokenize,mwt,pos')
def pos_tager(text):
  doc = nlp_pos(text)
  pos = [word.xpos for sent in doc.sentences for word in sent.words if word.text not in ['ش','شان','م', 'مان', 'ند','ست','یت', 'تان', 'ت','اش']]
  # pos_ = [(word.text , word.upos, word.xpos) for sent in doc.sentences for word in sent.words ]
  # print(pos_)
  return pos

In [9]:
from tqdm import tqdm

In [ ]:
def check_consist(pos_tag, kasre_tag, sent):
  l1 = len(pos_tag)
  l2 = len(kasre_tag)
  l3 = len(sent.split())
  if l1!=l2:
    return False
  if l1!=l3:
    return False
  if l3!=l2:
    return False
  return True

In [ ]:
with open('data/train_data.txt', encoding='utf-8') as f:
    data = f.readlines()
sents = list()
kasre_tags = list()
temp_sent = ''
temp_tags = list()
pos_tags = list()
for i, line in tqdm(enumerate(data)):
    # if i>10000: break
    try:
        word, ez = line.split()
        word += ' '
        temp_sent += word
        temp_tags.append(ez)
        if word in ['. ', '# ']:
            pos_tag = pos_tager(temp_sent)
            ok = check_consist(pos_tag, temp_tags, temp_sent)
            if ok:
              sents.append(temp_sent) 
              kasre_tags.append(temp_tags)
              pos_tags.append(pos_tag)
            temp_sent = ''
            temp_tags = list()
    except:
        pos_tag = pos_tager(temp_sent)
        ok = check_consist(pos_tag, temp_tags, temp_sent)
        if ok:
          sents.append(temp_sent) 
          kasre_tags.append(temp_tags)
          pos_tags.append(pos_tag)
        temp_sent = ''
        temp_tags = list()   

130it [00:00, 517.04it/s]/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords
8612838it [1:46:31, 1347.55it/s]


In [ ]:
len(sents), len(kasre_tags), len(pos_tags)

(295721, 295721, 295721)

In [ ]:
raws = list()

for i in range(len(pos_tags)):
  for j in range(len(pos_tags[i])):
    raw = f'{sents[i].split()[j]}\t{pos_tags[i][j]}\t{kasre_tags[i][j]}'
    raws.append(raw)
  raws.append('\n')

In [ ]:
raws_text = '\n'.join(raws)
with open('data/train_clean.tsv', 'w') as f:
  f.write(raws_text)

In [ ]:
!cp data/train_clean.tsv drive/MyDrive/دیتاست\ بیجن\ خان/

In [10]:
# run this or bellow cell
!mkdir fastText
!cp drive/MyDrive/fastText/cc.fa.300.vec fastText/

In [11]:
!ls -sh fastText/cc.fa.300.vec

7.0G fastText/cc.fa.300.vec


In [ ]:
from fasttext import load_model

# original BIN model loading
f = load_model('fastText/cc.fa.300.bin')
lines=[]

# get all words from model
words = f.get_words()

with open('fastText/cc.fa.300.vec','w') as file_out:
    
    # the first line must contain number of total words and vector dimension
    file_out.write(str(len(words)) + " " + str(f.get_dimension()) + "\n")

    # line by line, you append vectors to VEC file
    for w in words:
        v = f.get_word_vector(w)
        vstr = ""
        for vi in v:
            vstr += " " + str(vi)
        try:
            file_out.write(w + vstr+'\n')
        except:
            pass

In [ ]:
!mkdir drive/MyDrive/fastText
!cp fastText/cc.fa.300.vec drive/MyDrive/fastText/

In [12]:
!mkdir checkpoints

In [ ]:
RUN = 0

In [3]:
class Config:
    def __init__(self):
        # directories
        self.train_data_dir = 'data/train_clean_p1.tsv'
        self.model_dir = 'models'
        self.we_model_dir = 'fastText/cc.fa.300.vec'
        self.we_pickled_model_dir = 'fastText/cc.fa.300.pickle'

        # general
        self.data_split = .1
        self.num_epochs = 25
        self.batch_size = 16
        self.shuffle_buffer = 320000
        self.num_tags = 6
        self.num_pos_tags = 33
        self.word_max_len = 30
        self.learning_rate = 1e-3
        self.max_len = 100

        # embeddings
        self.num_words = 100000
        self.word_embed_dim = 300
        self.num_chars = 256  # number of most frequent characters to be kept
        self.char_embed_dim = 32
        self.pos_embed_dim = 16

        # lstm variables
        self.lstm_units = 256  # number of hidden units in the RNN
        self.dropout = .5  # keeping probability

In [4]:
from tqdm import tqdm

In [16]:
!cp drive/MyDrive/indices.pickle ./

In [5]:
import re
import random
import pickle
from collections import Counter
import sys

import numpy as np
from gensim.models import KeyedVectors
from torch.utils.data import DataLoader, Dataset
import os
from torch.nn.functional import one_hot
import torch
import torch.nn as nn
import string
cfg = Config()


In [17]:

class KasreDS(Dataset):
    def __init__(self, cfg):
        # loading word embedding model
        try:
            handle = open(cfg.we_pickled_model_dir, 'rb') 
            self.word_embedding_model = pickle.load(handle)
        except FileNotFoundError:
            self.word_embedding_model = KeyedVectors.load_word2vec_format(cfg.we_model_dir, binary=False)
            with open(cfg.we_pickled_model_dir, 'wb') as handle:
                pickle.dump(self.word_embedding_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

        sents, all_pos_tags, all_ezafe_tags = self._data_reader(cfg.train_data_dir)
        print(len(sents))
        sents_shuf = []
        all_pos_tags_shuf = []
        all_ezafe_tags_shuf = []
        index_shuf = list(range(len(sents)))

        for i in index_shuf:
            sents_shuf.append(sents[i])
            all_pos_tags_shuf.append(all_pos_tags[i])
            all_ezafe_tags_shuf.append(all_ezafe_tags[i])

        random.seed(17)
        random.shuffle(index_shuf)
        
        self.sents_shuf = sents_shuf
        print('data:', len(self.sents_shuf[0]))
        self.data = sents_shuf
        sents, all_pos_tags, all_ezafe_tags = self._data_reader(cfg.train_data_dir)
        self.sents = sents
        self.all_pos_tags = all_pos_tags
        self.all_ezafe_tags = all_ezafe_tags

        
        try:
              with open('indices.pickle', 'rb')  as handle:
                self.char_to_index, self.word_to_index, self.pos_tag_to_index, self.ezafe_tag_to_index = pickle.load(handle)
                
                print(self.pos_tag_to_index)

                self.index_to_word = {i: key for key, i in self.word_to_index.items()}
                self.index_to_ezafe_tag = {i: key for key, i in self.ezafe_tag_to_index.items()}
          
              

        except FileNotFoundError:
            print('Building vocabulary...')

            vocab_list = []
            char_list = []
            for sent in self.data:
                for word in sent:
                    vocab_list.append(word)
                    for char in word:
                        char_list.append(char)
            
            most_common_words = Counter(vocab_list).most_common(cfg.num_words)
            most_common_chars = Counter(char_list).most_common(cfg.num_chars)
            
            self.word_to_index = {}
            for i, pair in enumerate([('<PAD>', 0)] + most_common_words):
                self.word_to_index[pair[0]] = i + 1

            self.char_to_index = {}
            for i, pair in enumerate([('<PAD>', 0), ('<UNK>', 1)] + most_common_chars):
                self.char_to_index[pair[0]] = i + 1
            
            self.pos_tag_to_index = {}
            for i, tag in enumerate(set(x for y in self.all_pos_tags for x in y)):
                self.pos_tag_to_index[tag] = i + 1

            self.ezafe_tag_to_index = {'O': 0, 'e': 1,'ye': 2, 've':3, 'y':4, '@e':5}

            self.index_to_word = {i: key for key, i in self.word_to_index.items()}
            self.index_to_ezafe_tag = {i: key for key, i in self.ezafe_tag_to_index.items()}

            # saving the tokenizers
            with open('indices.pickle', 'wb') as handle:
                indices = self.char_to_index, self.word_to_index, self.pos_tag_to_index, self.ezafe_tag_to_index
                pickle.dump(indices, handle, protocol=pickle.HIGHEST_PROTOCOL)


    def _data_reader(self, directory):
        sents, sent = [], []
        all_ezafe_tags, ezafe_tags = [], []
        all_pos_tags, pos_tags = [], []
        with open(directory) as bijankhan_corpus:
            for line in bijankhan_corpus:
                if line != '\n':
                    word, pos_tag, ezafe_tag = line.strip().split('\t')
                    sent.append(word.replace('ي', 'ی').replace('ك', 'ک').replace('ة', 'ه'))
                    pos_tags.append(pos_tag)
                    ezafe_tags.append(ezafe_tag)
                else:
                    if len(sent)>1 :
                      sents.append(sent)
                      all_pos_tags.append(pos_tags)
                      all_ezafe_tags.append(ezafe_tags)
                     
                    sent = []
                    pos_tags = []
                    ezafe_tags = []

        return sents, all_pos_tags, all_ezafe_tags
	

    def _pad(self, word):
        for _ in range(cfg.word_max_len - len(word)):
            word.append(0)
        return word
    

    def _sent_to_index(self, sentence, mode='word'):
        if mode is 'word':
            return [self.word_to_index.get(word, 1) for word in sentence]
        elif mode is 'char':
            indexed_sentence = []
            for word in sentence:
                indexed_word = []
                for char in word:
                    indexed_word.append(self.char_to_index.get(word, 1))
                indexed_sentence.append(self._pad(indexed_word))
            return indexed_sentence


    def _sent_to_embed(self, sentence):
        embed_sent = []
        for word in sentence:
            try:
                embed_sent.append(self.word_embedding_model[word])
            except KeyError:
                embed_sent.append([0 for _ in range(cfg.word_embed_dim)])
        return embed_sent

    
    def _pos_tags_to_index(self, tags):
        return torch.stack([one_hot(torch.tensor(self.pos_tag_to_index[tag]), num_classes=cfg.num_pos_tags+1) for tag in tags])

    
    def _ezafe_tags_to_index(self, tags):
        return [self.ezafe_tag_to_index[tag] for tag in tags]

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        char = True
        pos=None
        sent, pos_tag, ezafe_tag = self.sents[idx], self.all_pos_tags[idx], self.all_ezafe_tags[idx]
        sent_char = self._sent_to_index(sent, mode='char')
        sent_word = self._sent_to_embed(sent)
        length = [1 for _ in range(len(sent))]
        tag = self._ezafe_tags_to_index(ezafe_tag)
        pos_tag = self._pos_tags_to_index(pos_tag)
        pos_tag = torch.tensor(pos_tag)
        # if char:
        #     return (np.array(sent_word), np.array(sent_char), np.array(length)), np.array(tag)
        if char:
            return torch.tensor(sent_word), pos_tag, torch.tensor(tag)


In [6]:

class KasreTokenizer:
  def __init__(self, cfg, pos_pipe, normalizer):
    self.normalizer = normalizer
    self.pos_pipe = pos_pipe
    punctuations = r')(}{:؟!،؛»«.' + r"/<>?.,:;"
    self.punctuations = '[' + punctuations + string.punctuation + ']'
    handle = open(cfg.we_pickled_model_dir, 'rb') 
    self.word_embedding_model = pickle.load(handle)
    with open('indices.pickle', 'rb')  as handle:
      self.char_to_index, self.word_to_index, self.pos_tag_to_index, self.ezafe_tag_to_index = pickle.load(handle)
      self.index_to_word = {i: key for key, i in self.word_to_index.items()}
      self.index_to_ezafe_tag = {i: key for key, i in self.ezafe_tag_to_index.items()}

  def _sent_to_index(self, sentence, mode='word'):
      if mode is 'word':
          return [self.word_to_index.get(word, 1) for word in sentence]
      elif mode is 'char':
          indexed_sentence = []
          for word in sentence:
              indexed_word = []
              for char in word:
                  indexed_word.append(self.char_to_index.get(word, 1))
              indexed_sentence.append(self._pad(indexed_word))
          return indexed_sentence


  def _sent_to_embed(self, sentence):
      embed_sent = []
      for word in sentence:
          try:
              embed_sent.append(self.word_embedding_model[word])
          except KeyError:
              embed_sent.append([0 for _ in range(cfg.word_embed_dim)])
      return embed_sent

  
  def _pos_tager(self, text):
    doc = self.pos_pipe(text)
    pos = list()
    for i in range(len(doc.sentences)):
      for j in range(len(doc.sentences[i].tokens)):
        text = doc.sentences[i].tokens[j].words[0].text
        xpos = doc.sentences[i].tokens[j].words[0].xpos
        pos.append(xpos)
    return pos

  def _pos_tags_to_index(self, tags):
      return torch.stack([one_hot(torch.tensor(self.pos_tag_to_index[tag]), num_classes=cfg.num_pos_tags+1) for tag in tags])
  
  def _sent_to_pos(self, sent):
      pos = self._pos_tager(sent)
      return pos

  def tokenizer(self, sent):
      sent = self.normalizer.normalize(sent)
      sent = re.sub(self.punctuations, lambda x: ' ' + x.group() + ' ', sent)
      sent_ = sent.split()
      sent_word = self._sent_to_embed(sent_)
      pos_tag = self._sent_to_pos(sent)
      pos_tag = self._pos_tags_to_index(pos_tag)
      pos_tag = torch.tensor(pos_tag)
      return torch.tensor(sent_word), pos_tag, sent_

In [18]:
kasre_ds = KasreDS(cfg)

127229
data: 23
{'N_IANM': 1, 'PRENUM': 2, 'ADJ_AJSUP': 3, 'PART': 4, 'PSUS': 5, 'PR_UCREFX': 6, 'PREM_DEMAJ': 7, 'ADJ_AJP': 8, 'PR_DEMON': 9, 'V_ACT': 10, 'PREP': 11, 'SUBR': 12, 'PR_INTG': 13, 'PR_RECPR': 14, 'AUX_PASS': 15, 'IDEN': 16, 'V_MODL': 17, 'ADV_SADV': 18, 'PUNC': 19, 'PREM_QUAJ': 20, 'PR_JOPER': 21, 'PREM_AMBAJ': 22, 'PR_CREFX': 23, 'ADR_PRADR': 24, 'N_ANM': 25, 'ADR_POSADR': 26, 'CONJ': 27, 'POSTP': 28, 'PR_SEPER': 29, 'V_PASS': 30, 'ADJ_AJCM': 31, 'AUX': 32, 'POSNUM': 33}


In [ ]:
!cp indices.pickle drive/MyDrive/

In [ ]:
len(kasre_ds)

127229

In [ ]:
_, x, _ = kasre_ds[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
train_dataset, val_dataset = torch.utils.data.random_split(kasre_ds,(126229, 1000), generator=torch.Generator().manual_seed(42) )

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=True, 
)

test_loader = DataLoader(
    val_dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True
)

In [ ]:
dataloaders = {'train': train_loader,
               'test': test_loader,
               'val': val_loader}

In [8]:
class KasreAdder(nn.Module):
  def __init__(self,cfg):
    super().__init__()
    self.cfg = cfg
    self.lstm = nn.LSTM(cfg.word_embed_dim, cfg.lstm_units // 2,
                            num_layers=2, bidirectional=True, batch_first=True)
    
    self.lstm_pos = nn.LSTM(34, 8,
                            num_layers=1, bidirectional=True, batch_first=True)
    
    self.hidden2tag = nn.Linear(cfg.lstm_units+16, cfg.num_tags)
    self.dropout = nn.Dropout()
    self.hidden = self.init_hidden()

  def init_hidden(self):
    return (torch.zeros(4, 1, self.cfg.lstm_units // 2).to(device),
            torch.zeros(4, 1, self.cfg.lstm_units // 2).to(device))

  def init_hidden_pos(self):
    return (torch.zeros(2, 1, 8).to(device),
            torch.zeros(2, 1, 8).to(device)) 
  
    
  def forward(self, x, pos, hidden=None):
    # print(f'x shape {x.shape}')
    self.hidden = hidden
    if hidden is None:
      self.hidden = self.init_hidden()
      self.hidden_pos = self.init_hidden_pos()
    lstm_out, self.hidden = self.lstm(x, self.hidden,)
    lstm_pos_out, self.hidden_pos = self.lstm_pos(pos, self.hidden_pos,)
    # print(f'lstm_out {lstm_out.shape}')
    # lstm_out = lstm_out.view(len(x), self.cfg.lstm_units)
    lstm_out = torch.concat((lstm_out, lstm_pos_out), dim=2)
    lstm_out = self.dropout(lstm_out)
    lstm_feats = self.hidden2tag(lstm_out[0])
    return lstm_feats
    

In [9]:
model = KasreAdder(cfg).to(device)

In [10]:
model.load_state_dict(torch.load('drive/MyDrive/kasre_pos_model_big.pt'))

<All keys matched successfully>

In [ ]:
lr=1e-3
epochs = 2
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, threshold=0.05, min_lr=1e-7, verbose=True, factor=0.1)
es_patience = 50

In [ ]:
def train(model, optimizer, criterion, lr_scheduler, dataloaders, num_epochs=10):
  model = model.train()
  train_loss_history = list()
  val_loss_history = list()
  best_epoch = {'epoch':0, 'accuracy':0, 'loss':-100.0}
  model_path = f'checkpoints/RUN_{RUN}'
  os.mkdir(model_path)
  for epoch in range(num_epochs):
    model = model.train()
    pbar = tqdm(dataloaders['train'])
    epoch_losses = np.array([])
    for (batch, pos_batch, labels) in pbar:
      batch = batch.to(device, dtype=torch.float)
      pos_batch = pos_batch.to(device, dtype=torch.float)
      labels = labels.to(device, dtype=torch.long)[0]
      optimizer.zero_grad()
      out = model(batch, pos_batch)
      # print(batch.shape)
      # print(labels.shape)
      # print('*************')
      # print(out.shape)
      loss = criterion(out, labels)
      loss.backward()
      optimizer.step()
      epoch_losses = np.append(epoch_losses, loss.item())
      pbar.set_description(f'epoch {epoch+1}/{num_epochs},\
                            loss= {epoch_losses.mean():.4f}')
    train_loss_history.append(epoch_losses.mean())
    # lr_scheduler.step()

    # validation
    model = model.eval()
    with torch.no_grad():
      pbar = tqdm(dataloaders['val'])
      val_loss = np.array([])
      n_samples = 0
      n_corrects = 0
      for (batch, pos_batch, labels) in pbar:
        batch = batch.to(device, dtype=torch.float)
        pos_batch = pos_batch.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)[0]
        # labels = torch.tensor([l for l in labels for _ in range(30)]).to(device, dtype=torch.long)
        out = model(batch, pos_batch)
        loss = criterion(out, labels)
        val_loss = np.append(val_loss, loss.item())
        out = nn.Softmax()(out)
        out = out.argmax(1)
        n_corrects += (out == labels).sum()
        n_samples += out.shape[0]
      val_acc = n_corrects/n_samples
      val_loss = val_loss.mean()
      val_loss_history.append(val_loss)
      lr_scheduler.step(val_loss)
      if val_acc > best_epoch['accuracy']:
        best_epoch = {'epoch':epoch, 'accuracy':val_acc, 'loss':val_loss}
        best_model_path = os.path.join(model_path, f'best_val_checkpoint.pt')
        torch.save(model.state_dict(), best_model_path)
      print(f'epoch {epoch+1}/{num_epochs}, loss:= {val_loss.mean():.4f}, accuracy= {(n_corrects/n_samples):.4f}')
      pbar.set_description(f'epoch {epoch+1}/{num_epochs}, loss:= {val_loss.mean():.4f}, accuracy= {(n_corrects/n_samples):.4f}')
    if (sorted(val_loss_history[-es_patience:]) == val_loss_history[-es_patience:]) and (len(val_loss_history)>2*es_patience):
      break
  model.load_state_dict(torch.load(best_model_path))
  print(best_epoch)
  return model, train_loss_history, val_loss_history

In [ ]:
RUN += 1
model, loss_history, val_loss_history = train(model, optimizer, criterion, lr_scheduler, dataloaders, epochs)

  0%|          | 0/126229 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:175: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
epoch 1/2,                            loss= 0.0805: 100%|██████████| 126229/126229 [37:49<00:00, 55.63it/s]
100%|██████████| 1000/1000 [00:05<00:00, 173.65it/s]


epoch 1/2, loss:= 0.0573, accuracy= 0.9821


epoch 2/2,                            loss= 0.0475: 100%|██████████| 126229/126229 [37:41<00:00, 55.81it/s]
100%|██████████| 1000/1000 [00:05<00:00, 174.48it/s]


epoch 2/2, loss:= 0.0493, accuracy= 0.9855
{'epoch': 1, 'accuracy': tensor(0.9855, device='cuda:0'), 'loss': 0.04928675359127169}


In [ ]:
RUN

1

In [ ]:
!cp checkpoints/RUN_1/best_val_checkpoint.pt drive/MyDrive/kasre_pos_model_big.pt

In [ ]:
model.load_state_dict(torch.load('drive/MyDrive/kasre_model.pt'))
model.eval()

In [11]:
model.eval()

KasreAdder(
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, bidirectional=True)
  (lstm_pos): LSTM(34, 8, batch_first=True, bidirectional=True)
  (hidden2tag): Linear(in_features=272, out_features=6, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [12]:
from parsivar import Normalizer
import re

In [13]:
my_normalizer = Normalizer()
tokenizer = KasreTokenizer(cfg, pos_pipe=nlp_pos, normalizer=my_normalizer)

In [19]:
tag2kasre =  {0:'',
              1:'ِ',
              2:'‌یِ',
              3:'ِ',
              4:'ی',
              5:'ِ'}

def add_kasre(sent):
  sent_words, pos, sent_ = tokenizer.tokenizer(sent)
  # print(sent_)
  sent_words = torch.tensor(sent_words).float().to(device)
  pos = torch.tensor(pos).float().to(device)
  # print(f'sent shape {sent_words.shape}, pos shape {pos.shape}')
  with torch.no_grad():
    out = model(sent_words.unsqueeze(0), pos.unsqueeze(0))
    out = nn.Softmax()(out)
    out = out.argmax(1).cpu().numpy()
  result = ''
  print(out)
  for i,word in enumerate(sent_):
    ezafe_tag = tag2kasre[out[i]]
    if (word[-1] in ['ی']) and out[i]==2:
      ezafe_tag = tag2kasre[1]
    result += word + ezafe_tag + ' '
  return result




In [20]:
import time

In [21]:
start = time.time()
print(add_kasre('در پنجاه سالگی نیز سرانجام به خاطر همین ازهم پاشیدگی هویتی دچار بحرانی عمیق شد'))
end = time.time()
print(end - start)

[0 0 0 0 0 0 1 0 0 1 0 1 2 0 0]
در پنجاه سالگی نیز سرانجام به خاطرِ همین ازهم پاشیدگیِ هویتی دچارِ بحرانیِ عمیق شد 
0.053612709045410156


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated

In [18]:
f = open('sample_text.txt', 'r')
for line in f:
  print(add_kasre(line))
  print(line)
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated

تفسیرِ 
تفسیر

جین دو بلسیل عطشِ زیادی برای جلبِ توجه داشت 
جین دو بلسیل عطش زیادی برای جلب توجه داشت

او والدینش را خسته کرده‌بود ، طوری که مجبور شدند او را به صومعه‌ای در شهرِ پواتیه بفرستند 
او والدینش را خسته کرده بود، طوری که مجبور شدند او را به صومعه ای در شهر پواتیه بفرستند

او در آنجا با خودخواهی‌ها و حسِ برتری اش‌یِ دیگر راهبه‌ها را به حدِ جنون رساند 
او در آنجا با خودخواهی ها و حس برتری اش دیگر راهبه ها را به حد جنون رساند

وقتی به لودون فرستاده شد ، تصمیم گرفت روشی متفاوت در پیش بگیرد تا به شهرتی که شدیدا به آن نیاز داشت ، دست یابد 
وقتی به لودون فرستاده شد، تصمیم گرفت روشی متفاوت در پیش بگیرد تا به شهرتی که شدیداً به آن نیاز داشت، دست یابد

وقتی به او کتاب‌هایی در زمینه‌یِ معنویت داده شد ، به این نتیجه رسید که از نظرِ دانش و عرفان از بقیه پیشی گیرد تا برتریِ خود را اثبات کند 
وقتی به او کتاب هایی در زمینۀ معنویت داده شد، به این نتیجه رسید که از نظر دانش و عرفان از بقیه پیشی گیرد تا برتری خود را اثبات کند

او در هر دو زمینه از خود درخشش نشان داد تا توانست نظرِ رئیسِ صومعه را

In [49]:
sent = 'انگار قرار است تا دم مرگ با هم مجادله داشته باشیم! او کتاب سونات کرویتسر را از دل همین تلخی های زندگی اش نوشت'
punctuations = r')(}{:؟!،؛»«.' + r"/<>?.,:;"
punctuations = '[' + punctuations + string.punctuation + ']'
sent = re.sub(punctuations, lambda x: ' ' + x.group() + ' ', sent)
sent

'انگار قرار است تا دم مرگ با هم مجادله داشته باشیم !  او کتاب سونات کرویتسر را از دل همین تلخی های زندگی اش نوشت'

In [44]:
punctuations

'\\)\\(}{:؟،؛»«\\./<>?.,:;!'

In [22]:
doc = nlp_pos('انگار قرار است تا دم مرگ با هم مجادله داشته باشیم.» او کتاب سونات کرویتسر را از دل همین تلخی های زندگی اش نوشت')

In [23]:
pos = list()
for i in range(len(doc.sentences)):
  for j in range(len(doc.sentences[i].tokens)):
    text = doc.sentences[i].tokens[j].words[0].text
    xpos = doc.sentences[i].tokens[j].words[0].xpos
    print(text, xpos)
    pos.append((text, xpos))

انگار PSUS
قرار N_IANM
است AUX
تا PREP
دم N_IANM
مرگ N_IANM
با PREP
هم PR_RECPR
مجادله N_IANM
داشته V_ACT
باشیم AUX
. PUNC
» PUNC
او PR_SEPER
کتاب N_IANM
سونات N_IANM
کرویتسر N_IANM
را POSTP
از PREP
دل N_IANM
همین PREM_DEMAJ
تلخی N_IANM
های N_IANM
زندگی N_IANM
اش PR_JOPER
نوشت V_ACT
